In [53]:
import requests
import pandas as pd
import json
import psycopg2
import time

# Get house data of Realtor by RapidAPI

## Step 1)
Invoke list API to get house list and store into the Postgresql

## Step 2
Invoke price history API to get history data, then to store into the Postgresql.

## Step 3

Save as CSV file and upload to Github

# Some basic database operation function

In [44]:
# connect postgresql
def get_postgresql_connect():
    conn = psycopg2.connect(host="127.0.0.1",user="postgres",password="123456",database="aim_new_final")
    return conn

def query_houses():
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from t_houses "
    cur.execute(sql)
    
    rows = cur.fetchall()
        
    conn.close()
    
    return rows

def insert_houses(house_list):
    conn= get_postgresql_connect()
    cur = conn.cursor()
    
    sql = """
        INSERT INTO public.t_houses(property_id, prop_type, beds, baths_full, full_json, county, state)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
    
    for h in house_list:
        
        asjson = json.dumps(h)
        
        if 'property_id' in h and 'prop_type' in h:
            
            beds = 0
            if 'beds' in h:
                beds = h['beds']
                
            baths_full = 0
            if 'baths_full' in h:
                baths_full = h['baths_full']
            
            county = ''
            state = ''
            
            if 'address' in h:
                address = h['address']
                
                if 'county' in address:
                    county = address['county']
                if 'state' in address:
                    state = address['state']
                
            values = (h['property_id'], h['prop_type'], beds, baths_full, asjson, county, state)
        

        cur.execute(sql, values)
        
    conn.commit()
    print("Records created successfully")
    conn.close()

    
def query_history_prices():
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from t_house_history_prices "
    cur.execute(sql)
    
    rows = cur.fetchall()
    
    conn.close()
    
    return rows
    
def insert_history_prices(property_id, history_list):
    conn= get_postgresql_connect()
    cur = conn.cursor()
    
    sql = """
        INSERT INTO t_house_history_prices(property_id, price, price_sqft, price_event, price_event_date, sqft)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    # {'event_name': 'Sold', 'date': '2020-12-10T17:00:00Z', 'price': 490000, 'price_changed': 0, 
    # 'sqft': 1148, 'datasource_name': 'StatenIsland', 'source': 'MLS #1138690'}
    for h in history_list:
        
        price_sqft = 0
        
        if h['sqft'] and h['price'] and h['sqft'] > 0:
            price_sqft = h['price'] / h['sqft']
        
        values = (property_id, h['price'], price_sqft, h['event_name'], h['date'], h['sqft'])

        cur.execute(sql, values)
        
    conn.commit()
    print("Records created successfully")
    conn.close()
#query_houses()

In [10]:
# convert a json string into a Python object using the json.loads() function
def save_to_db(json_str):
    
    all_json = json.loads(json_str)
    house_list = all_json['properties']
    
    insert_houses(house_list)

# save_to_db(json_test)

In [ ]:
rapidapi_key = "yuehao's rapidapi key has been hidden"

# Step 1)

## Invoke list API to get house list and store into the Postgresql

In [11]:
def find_list():
    url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"
    
    targets = [#{"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"multi_family","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"multi_family","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"multi_family","sort":"sold_date"}]
    
    # querystring = {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"condo","sort":"sold_date"}

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }
    
    for t in targets:
        print(t)
        querystring = t
        
        response = requests.request("GET", url, headers=headers, params=querystring)
        
        json_str = response.text
        
        save_to_db(json_str)
        print('save successful')
    
# find_list()

{'city': 'New York City', 'offset': '0', 'state_code': 'NY', 'limit': '30', 'prop_type': 'single_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'New York City', 'offset': '0', 'state_code': 'NY', 'limit': '30', 'prop_type': 'multi_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'condo', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'single_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'multi_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Houston', 'offset': '0', 'state_code': 'TX', 'limit': '30', 'prop_type': 'condo', 'sort': 'sold_date'}
Records created successfully
save successfu

In [55]:
# result['properties']

In [56]:
def read_house_detail_json(house_detail_json_str):
    try:
        detail_json = json.loads(house_detail_json_str)
        if detail_json and 'properties' in detail_json:

            p = detail_json['properties']
            if len(p) > 0:
                detail_dic = p[0]
                if 'property_id' in detail_dic and 'property_history' in detail_dic:
                    d_list = detail_dic['property_history']
                    if len(d_list) > 0:
                        insert_history_prices(detail_dic['property_id'], d_list)
    except:
      print("An exception occurred")

In [58]:

    
def find_house_detail(property_id):
    url = "https://realtor.p.rapidapi.com/properties/v2/detail"

    querystring = {"property_id":property_id}

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.text

house_detail_json = find_house_detail("O4684373068")


NameError: name 'rapidapi_key' is not defined

In [47]:
# test
#rows = query_houses()
#print(rows)

# Step 2

## Invoke price history API to get history data, then to store into the Postgresql.

In [54]:
def load_peoperty_detail():
    
    # 1) load db
    rows = query_houses()
    
    # 2) invoke web api
    i = 0
    if len(rows) > 0:
        for row in rows:
            property_id = row[1]
            print("NO. {}, id = {}".format(i, property_id))
            
            time.sleep(3)
        
            detail_json_str = find_house_detail(property_id)
            
            # 3) insert db
            if detail_json_str:
                read_house_detail_json(detail_json_str)
            
            
            i += 1
    
    

load_peoperty_detail()

NO. 0, id = O4684373068
Records created successfully
NO. 1, id = O4972388730
Records created successfully
NO. 2, id = O3944496635
Records created successfully
NO. 3, id = O3296087243
Records created successfully
NO. 4, id = O3484025688
Records created successfully
NO. 5, id = O3645991546
Records created successfully
NO. 6, id = O3075273785
Records created successfully
NO. 7, id = O4763549378
Records created successfully
NO. 8, id = O3341300981
Records created successfully
NO. 9, id = O9064118988
Records created successfully
NO. 10, id = O3498573255
Records created successfully
NO. 11, id = O1342399516
Records created successfully
NO. 12, id = O2738107654
Records created successfully
NO. 13, id = O2017028114
Records created successfully
NO. 14, id = O1414670866
Records created successfully
NO. 15, id = O4770831613
Records created successfully
NO. 16, id = O3285464677
Records created successfully
NO. 17, id = O3770561258
Records created successfully
NO. 18, id = O9325484330
Records creat

Records created successfully
NO. 155, id = O2263450120
Records created successfully
NO. 156, id = O1589582768
Records created successfully
NO. 157, id = O1361458248
Records created successfully
NO. 158, id = O1542303659
Records created successfully
NO. 159, id = O1197045282
Records created successfully
NO. 160, id = O2423790248
Records created successfully
NO. 161, id = O2385973540
Records created successfully
NO. 162, id = O2308368178
Records created successfully
NO. 163, id = O1389945876
Records created successfully
NO. 164, id = O1000007648
Records created successfully
NO. 165, id = O2682055304
Records created successfully
NO. 166, id = O2923438706
Records created successfully
NO. 167, id = O2247767549
Records created successfully
NO. 168, id = O1157137309
Records created successfully
NO. 169, id = O1519498036
Records created successfully
NO. 170, id = O1387209868
Records created successfully
NO. 171, id = O9504223440
Records created successfully
NO. 172, id = O1964960837
Records cr

In [59]:
#print(detail_json['properties'])
#print(type(detail_json['properties']))

# Step 3

## Save as CSV file and upload to Github

In [61]:
# connect postgresql
def get_postgresql_connect():
    conn = psycopg2.connect(host="127.0.0.1",user="postgres",password="123456",database="aim_new_final")
    return conn

# query average price of three counties.
def query_house_avg():
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from v_house_avg "
    cur.execute(sql)
    
    rows = cur.fetchall()
        
    conn.close()
    
    return rows

house_avg_tuples = query_house_avg()

In [62]:
# rename the dataframe
house_avg_columns = ["property_id", "county", "state", "prop_type", "2020_avg", "2019_avg", 
                                                    "2020_01_avg", "2020_02_avg", "2020_03_avg", "2020_04_avg", "2020_05_avg", 
                                                    "2020_06_avg", "2020_07_avg", "2020_08_avg", "2020_09_avg", 
                                                    "2020_10_avg", "2020_11_avg", "2020_12_avg"]

house_avg_df = pd.DataFrame(house_avg_tuples, columns = house_avg_columns) 
  
house_avg_df.head(3)  

,property_id,county,state,prop_type,2020_avg,2019_avg,2020_01_avg,2020_02_avg,2020_03_avg,2020_04_avg,2020_05_avg,2020_06_avg,2020_07_avg,2020_08_avg,2020_09_avg,2020_10_avg,2020_11_avg,2020_12_avg
0,O1000007648,Los Angeles,California,multi_family,376.38,95.34,376.38,376.38,376.38,376.38,376.38,376.38,376.38,376.38,376.38,376.38,378.63,378.63
1,O1028836967,Los Angeles,California,single_family,718.43,51.85,718.43,718.43,718.43,718.43,718.43,718.43,718.43,718.43,718.43,718.43,722.56,722.56
2,O1032600397,Los Angeles,California,condo,622.91,530.76,622.91,622.91,622.91,622.91,622.91,622.91,622.91,622.91,622.91,622.91,613.34,613.34


In [63]:
# write into csv
house_avg_df.to_csv('./data/final_house_3cities.csv', index=True)